In [1]:
import os
cwd_old = os.getcwd()
os.chdir('..')
cwd_new = os.getcwd()
print("The working directory is moved from {} to {}.".format(cwd_old, cwd_new))

The working directory is moved from /data2/sungjaecho/Projects/tacotron2/dev_ipynb to /data2/sungjaecho/Projects/tacotron2.


# Split DB with respect to (speaker, emotion) pairs

In [2]:
import os
import pandas as pd
import random

In [3]:
db_csv_path = os.path.join('metadata', 'ketts2.csv'); print(db_csv_path)

metadata/ketts2.csv


In [4]:
df_ketts2 = pd.read_csv(db_csv_path)
df_ketts2.head()

,database,split,wav_path,duration,text,speaker,sex,emotion,lang
0,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,5.841,새로 익히고 있는 증권업무도 전혀 낯선 일입니다.,ketts2-40m,m,fearful,ko
1,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,4.885,소원을 들어주는 요술램프가 있었으면 좋겠다.,ketts2-50f,w,angry,ko
2,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,5.303,우리 앞에 어떤 가능성도 배제하지 말자.,ketts2-60f,w,sad,ko
3,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,4.458,나는 정도를 걸으면서 양국은 하나라는 생각을 응원한다.,ketts2-30f,w,neutral,ko
4,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,6.550,무엇이든 가장 보편적인 것이 사람들에게 사랑받는 법이다.,ketts2-40m,m,neutral,ko


In [5]:
speaker = 'ketts2-40m'
emotion = 'neutral'
df_selected = df_ketts2[(df_ketts2.speaker == speaker) & (df_ketts2.emotion == emotion)]
df_selected.head()

,database,split,wav_path,duration,text,speaker,sex,emotion,lang
4,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,6.550,무엇이든 가장 보편적인 것이 사람들에게 사랑받는 법이다.,ketts2-40m,m,neutral,ko
9,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,5.491,주쿠웨이트 한국대사관의 기능은 정상적인가.,ketts2-40m,m,neutral,ko
71,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,8.022,여기다 이번에 선정된 각 그룹별 주력기업들은 재무구조가 아주 나쁘다.,ketts2-40m,m,neutral,ko
73,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,4.906,나는 매일 자기전 아이들에게 동화책을 읽어준다.,ketts2-40m,m,neutral,ko
94,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,6.440,사고로 인해 열차 운행 시각이 모조리 바뀌어 승객들의 혼란을 빚었다.,ketts2-40m,m,neutral,ko


In [6]:
indexes = df_selected.index.tolist()

In [7]:
df_ketts2.at[indexes, 'split'] ='that'

In [8]:
df_selected = df_ketts2[(df_ketts2.speaker == speaker) & (df_ketts2.emotion == emotion)]
df_selected.head()

,database,split,wav_path,duration,text,speaker,sex,emotion,lang
4,ketts2,that,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,6.550,무엇이든 가장 보편적인 것이 사람들에게 사랑받는 법이다.,ketts2-40m,m,neutral,ko
9,ketts2,that,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,5.491,주쿠웨이트 한국대사관의 기능은 정상적인가.,ketts2-40m,m,neutral,ko
71,ketts2,that,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,8.022,여기다 이번에 선정된 각 그룹별 주력기업들은 재무구조가 아주 나쁘다.,ketts2-40m,m,neutral,ko
73,ketts2,that,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,4.906,나는 매일 자기전 아이들에게 동화책을 읽어준다.,ketts2-40m,m,neutral,ko
94,ketts2,that,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,6.440,사고로 인해 열차 운행 시각이 모조리 바뀌어 승객들의 혼란을 빚었다.,ketts2-40m,m,neutral,ko


# Implementation

In [9]:
#split_ratio = {'train':0.95, 'val':0.025, 'test':0.025}
split_ratio = {'train':0.95, 'val':10, 'test':10}
speech_classes = ['speaker', 'emotion']

df_unique_classes = df_ketts2[speech_classes].drop_duplicates(subset = speech_classes)
n_unique_classes = df_unique_classes.shape[0]

for i, row in df_unique_classes.iterrows():
    df_selected = df_ketts2[(df_ketts2.speaker == row.speaker) & (df_ketts2.emotion == row.emotion)]
    
    df_selected_len = df_selected.shape[0]
    
    if split_ratio['val'] < 1:
        i_val_start = int(df_selected_len * split_ratio['train'])
        i_test_start = int(df_selected_len * (split_ratio['train'] + split_ratio['val']))
    else:
        # This case specifies split ratio using the number of samples
        i_val_start = df_selected_len - split_ratio['val'] - split_ratio['test']
        i_test_start = df_selected_len - split_ratio['test']
        
    n_train = i_val_start
    n_val = i_test_start - i_val_start
    n_test = df_selected_len - i_test_start
    
    split_labels = (['train'] * n_train) + (['val'] * n_val) + (['test'] * n_test)
    
    random.seed(3141)
    random.shuffle(split_labels)
    
    indexes = df_selected.index.tolist()
    df_ketts2.at[indexes, 'split'] = split_labels

# Statistics for proof

In [10]:
df_ketts2.groupby(by=['speaker', 'emotion', 'split']).count()

database  wav_path  duration  text  sex  lang
speaker    emotion   split                                               
ketts2-20m angry     test         10        10        10    10   10    10
                     train       480       480       480   480  480   480
                     val          10        10        10    10   10    10
           disgusted test         10        10        10    10   10    10
                     train       480       480       480   480  480   480
...                              ...       ...       ...   ...  ...   ...
ketts2-60f sad       train       480       480       480   480  480   480
                     val          10        10        10    10   10    10
           surprised test         10        10        10    10   10    10
                     train       480       480       480   480  480   480
                     val          10        10        10    10   10    10

[126 rows x 6 columns]

In [11]:
db_csv_path = os.path.join('metadata', 'ketts.csv'); print(db_csv_path)

metadata/ketts.csv


In [12]:
df_ketts = pd.read_csv(db_csv_path)
df_ketts.head()

,database,split,wav_path,duration,text,speaker,sex,emotion,lang
0,ketts,train,/data2/sungjaecho/data_tts/KETTS/KETTS/wav/ket...,4.740,천안문사건 이후 양국간의 경제협력이 둔화된 것은 사실이나 서서히 회복되고 있고 북경...,ketts-30f,w,sad,ko
1,ketts,train,/data2/sungjaecho/data_tts/KETTS/KETTS/wav/ket...,5.506,그렇게 볼 수 있는 시각도 있을 것입니다.,ketts-30m,m,sad,ko
2,ketts,train,/data2/sungjaecho/data_tts/KETTS/KETTS/wav/ket...,4.271,특히 중소기업을 중심으로 한국과 경제협력이 이뤄지길 기대한다.,ketts-30m,m,happy,ko
3,ketts,train,/data2/sungjaecho/data_tts/KETTS/KETTS/wav/ket...,7.007,얼마전 공보처 신문과에 기자가 되고싶다는 어떤 젊은이가 전화를 해 온 적이 있다.,ketts-30m,m,fearful,ko
4,ketts,train,/data2/sungjaecho/data_tts/KETTS/KETTS/wav/ket...,4.271,나는 할리우드 영화에 나오는 사람 같은 멋진 군인이 되고 싶다.,ketts-30m,m,neutral,ko


In [15]:
df_ketts.speaker.unique().tolist()

['ketts-30f', 'ketts-30m']

In [16]:
df_ketts.emotion.unique().tolist()

['sad', 'happy', 'fearful', 'neutral', 'disgusted', 'angry', 'surprised']

In [3]:
db_csv_path = os.path.join('metadata', 'ketts2.csv'); print(db_csv_path)

metadata/ketts2.csv


In [4]:
df_ketts2 = pd.read_csv(db_csv_path)
df_ketts2.head()

,database,split,wav_path,duration,text,speaker,sex,emotion,lang
0,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,5.841,새로 익히고 있는 증권업무도 전혀 낯선 일입니다.,ketts2-40m,m,fearful,ko
1,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,4.885,소원을 들어주는 요술램프가 있었으면 좋겠다.,ketts2-50f,w,angry,ko
2,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,5.303,우리 앞에 어떤 가능성도 배제하지 말자.,ketts2-60f,w,sad,ko
3,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,4.458,나는 정도를 걸으면서 양국은 하나라는 생각을 응원한다.,ketts2-30f,w,neutral,ko
4,ketts2,train,/data2/sungjaecho/data_tts/KETTS2/KETTS2/wav/k...,6.550,무엇이든 가장 보편적인 것이 사람들에게 사랑받는 법이다.,ketts2-40m,m,neutral,ko


In [17]:
df_ketts.speaker.unique().tolist()

['ketts-30f', 'ketts-30m']

In [18]:
df_ketts.emotion.unique().tolist()

['sad', 'happy', 'fearful', 'neutral', 'disgusted', 'angry', 'surprised']